# Virtual Sites

This notebook describes the use of virtual sites in OpenFF force fields.

In [ ]:
from openff.toolkit import ForceField

sage_with_example_virtual_sites = ForceField(
    "openff-2.2.0.offxml",
    "inputs/example-vsites-parameters-forcefield.offxml",
)

In [ ]:
import mdtraj
import nglview
import openmm
import openmm.unit
from openff.toolkit import Molecule

def _add_vsite_reps(widget, interchange):
    widget.clear_representations()
    atoms_with_vsites = [*interchange.to_openmm_topology().atoms()]
    if len(atoms_with_vsites) == interchange.topology.n_atoms + 1:
        vsite = next(iter(interchange["VirtualSites"].key_map))
    
        atom1 = vsite.orientation_atom_indices[0]
        name = vsite.name
    
        atom2 = [atom.index for atom in atoms_with_vsites if atom.name == name][0]
    
        widget.add_distance(atomPair=[[f"@{atom1}", f"@{atom2}"]])

    vsite_idcs = [
        str(v.index) for v in atoms_with_vsites[interchange.topology.n_atoms:]
    ]
    not_vsite_idcs = [
        str(v.index) for v in atoms_with_vsites[:interchange.topology.n_atoms]
    ]

    widget.add_licorice(selection="@" + ",".join(not_vsite_idcs))
    widget.add_representation("ball+stick", selection="@" + ",".join(vsite_idcs))

    return widget
        
def viz(
    smiles: str,
    force_field: ForceField = sage_with_example_virtual_sites,
) -> nglview.NGLWidget:
    molecule = Molecule.from_smiles(smiles, allow_undefined_stereo=True)
    molecule.generate_conformers(n_conformers=1)

    interchange = force_field.create_interchange(molecule.to_topology())
    
    w = interchange.visualize(backend='nglview', include_virtual_sites=True)
    return _add_vsite_reps(w, interchange)


def run(
    smiles: str,
    name: str,
    force_field: ForceField = sage_with_example_virtual_sites,
) -> nglview.NGLWidget:
    molecule = Molecule.from_smiles(smiles, allow_undefined_stereo=True)
    molecule.generate_conformers(n_conformers=1)

    interchange = force_field.create_interchange(molecule.to_topology())

    integrator = openmm.LangevinIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        1 * openmm.unit.femtoseconds,
    )

    simulation = interchange.to_openmm_simulation(integrator=integrator)

    dcd_reporter = openmm.app.DCDReporter("trajectory.dcd", 10)
    simulation.reporters.append(dcd_reporter)

    simulation.context.computeVirtualSites()
    simulation.minimizeEnergy()
    simulation.context.setVelocitiesToTemperature(integrator.getTemperature())
    simulation.step(1000)

    interchange.positions = simulation.context.getState(
        getPositions=True
    ).getPositions()

    open(f"{name}.xml", "w").write(
        openmm.XmlSerializer.serialize(interchange.to_openmm())
    )

    w = nglview.show_mdtraj(
        mdtraj.load(
            "trajectory.dcd",
            top=mdtraj.Topology.from_openmm(
                interchange.to_openmm_topology(),
            ),
        )
    )
    return _add_vsite_reps(w, interchange)


## `BondCharge` virtual site

This supports placement of a virtual site S along a vector between two specified atoms, e.g. to allow for a sigma hole for halogens or similar contexts. With positive values of the distance, the virtual site lies outside the first indexed atom (cyan in this image).

![](images/vsite_bondcharge.jpg)

In [ ]:
sage_with_example_virtual_sites["VirtualSites"].get_parameter({"type": "BondCharge"})[
    0
].to_dict()

In [ ]:
run("CCCCCl", name="sigma_hole")

## `MonovalentLonePair` virtual site

This is originally intended for situations like a carbonyl, and allows placement of a virtual site `S` at a specified distance `d` from the first indexed atom, with angles `inPlaneAngle` (θ₁ in the diagram) and `outOfPlaneAngle` (θ₂ in the diagram) relative to the plane defined by the three indexed atoms. The atom labeled `w` is not used to place the virtual site.

![](images/vsite_monovalent.jpg)

In [ ]:
sage_with_example_virtual_sites["VirtualSites"].get_parameter(
    {"type": "MonovalentLonePair"}
)[0].to_dict()

In [ ]:
run("COC1=C(C=CC(=C1)C=O)O", name="planar_carbonyl")

## `DivalentLonePair` virtual site

This is suitable for cases like four-point and five-point water models as well as pyrimidine; a virtual site `S` lies a specified distance `d` from the atom indexed 1 (blue) along the bisector of the angle between the three indexed atoms (if `outOfPlaneAngle` is zero) or out of the plane by the specified angle (if `outOfPlaneAngle` is nonzero) with its projection along the bisector. For positive values of the distance `d` the virtual site lies outside the 2-1-3 angle and for negative values it lies inside.

![](images/vsite_divalent.jpg)

In [ ]:
ForceField("tip4p_fb.offxml")["VirtualSites"].get_parameter({"type": "DivalentLonePair"})[
    0
].to_dict()


In [ ]:
run("O", name="tip4p_fb", force_field=ForceField("tip4p_fb.offxml"))

In [ ]:
ForceField("tip5p.offxml")["VirtualSites"].get_parameter({"type": "DivalentLonePair"})[
    0
].to_dict()


In [ ]:
run("O", name="tip5p", force_field=ForceField("tip5p.offxml"))

## `TrivalentLonePair` virtual site

This is suitable for planar or tetrahedral nitrogen lone pairs; a charge site `S` lies above the central atom (blue) a distance `d` along the vector perpendicular to the plane of the other three indexed atoms (2,3,4). With positive values of `d` the site lies above the central atom and with negative values it lies below.

![](images/vsite_trivalent.jpg)

In [ ]:
sage_with_example_virtual_sites["VirtualSites"].get_parameter(
    {"type": "TrivalentLonePair"}
)[0].to_dict()

In [ ]:
run("N", name="trivalent_nitrogen")

## Multiple virtual sites

Each of the molecules shown so far was selected to only have a single virtual site added, but larger molecules may have multiple copies of different types of virtual sites added.

In [ ]:
run("c1(Cl)c(Cl)c(CCCN)c(Cl)c(Cl)c1CCC(=O)", "many_vsites")